In [0]:
%sql
-- Unity catalog : initializes the workspace and schema.
USE CATALOG workspace;
USE SCHEMA default;

In [0]:
%sql
-- Data Preview : inspect the first few rows of the table.
SELECT * FROM workspace.default.ecommerce_bronze LIMIT 10;

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
2019-10-31 00:33:02 UTC,view,16700605,2053013559901684381,furniture.kitchen.chair,zeta,16.19,514196713,f29068b7-939c-4302-9701-068fb6391443
2019-10-31 00:33:03 UTC,view,16700256,2053013559901684381,furniture.kitchen.chair,null,13.10,513790869,ecd4159b-ae37-4ae0-9681-f18934bdbb21
2019-10-31 00:33:03 UTC,view,11700326,2053013554591695207,electronics.audio.acoustic,krk,362.69,522222104,b37f4d34-f360-4134-ac9d-75b7fb3bb63f
2019-10-31 00:33:04 UTC,view,4502748,2053013563877884791,appliances.kitchen.hob,indesit,244.77,545004802,25a87d66-ca29-34a5-61c4-22af0f949ab5
2019-10-31 00:33:05 UTC,view,26403432,2053013563651392361,null,null,109.66,521158963,1c0c8fac-12e6-44d3-b6b2-a22b16fd38e2
2019-10-31 00:33:06 UTC,view,5100572,2053013553341792533,electronics.clocks,apple,431.09,523227761,860b4189-d97f-49a5-a33a-69078a11723f
2019-10-31 00:33:06 UTC,view,42300027,2095518921321874323,furniture.living_room.cabinet,null,205.67,565855403,bd5190b6-52f4-4830-a5dd-849c15f2a161
2019-10-31 00:33:07 UTC,view,1480497,2053013561092866779,computers.desktop,apple,1785.12,559310552,6602b20d-df66-470f-8b48-f44fd52e3f44
2019-10-31 00:33:07 UTC,view,42200016,2095518917320508073,null,kartal,14.41,551896549,789175b3-b3b8-449f-bfeb-32e51b1f612c
2019-10-31 00:33:08 UTC,view,1004249,2053013555631882655,electronics.smartphone,apple,723.44,548271646,f0c67713-573f-42b5-add6-e960235ef24a


In [0]:
%sql
-- Inspect table metadata (managed vs external)
DESCRIBE EXTENDED workspace.default.ecommerce_bronze;

col_name,data_type,comment
event_time,string,null
event_type,string,null
product_id,string,null
category_id,string,null
category_code,string,null
brand,string,null
price,string,null
user_id,string,null
user_session,string,null
,,


In [0]:
%sql
-- Grant access to the actual data (Table)
GRANT SELECT ON TABLE workspace.default.ecommerce_bronze TO `lavy_kadali@yahoo.com`;

In [0]:
%sql
GRANT ALL PRIVILEGES ON TABLE workspace.default.ecommerce_bronze TO `lavy_kadali@yahoo.com`;  

In [0]:
%sql
-- Create a Controlled view on Gold Table
CREATE OR REPLACE VIEW ecommerce.vw_high_performance_products AS
SELECT
    product_id,
    price,
    SUM(price) AS revenue,
    COUNT(*) AS purchases
FROM workspace.default.ecommerce_bronze
GROUP BY product_id, price
HAVING COUNT(*) > 10;

In [0]:
%sql
GRANT SELECT ON VIEW ecommerce.vw_high_performance_products TO `lavy_kadali@yahoo.com`;

In [0]:
%sql
SELECT *
FROM ecommerce.vw_high_performance_products
LIMIT 10;

product_id,price,revenue,purchases
41100034,52.00,16068.0,309
16700287,308.86,268399.3400000004,869
1201525,64.33,14860.229999999994,231
5800572,27.00,2349.0,87
12720511,75.94,3265.4200000000005,43
15200054,79.80,10294.199999999999,129
1306133,1145.20,3378339.999999994,2950
26500655,112.23,16834.5,150
16400264,137.82,34592.81999999999,251
47900002,154.19,94826.85000000006,615


In [0]:
%sql
-- Finance focussed view
CREATE OR REPLACE VIEW ecommerce.vw_revenue_summary AS
SELECT
    product_id,
    SUM(CAST(price AS DOUBLE)) AS revenue
FROM workspace.default.ecommerce_bronze
GROUP BY product_id
HAVING SUM(CAST(price AS DOUBLE)) > 1000;